In [ ]:
import gensim
import pandas as pd
import smart_open
import random

# read data
#dataframe = pd.read_csv('../dataset/심판문제목요지.txt',delimiter='\t')
dataframe = pd.read_csv('./data/simpan_2018.csv')
dataframe.head()

In [ ]:
from gensim.models import word2vec
import codecs
from konlpy.tag import Okt

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
okt = Okt()

In [ ]:
def kotag(row):
    dem_str = row['dem_str']
    
    malist = okt.pos(row['note'], norm=True, stem=True)
    #print(malist)

    r = []
    for word, pumsa in malist:
        if pumsa == "Noun":     # 명사
            if len(word) > 1:   # 한글자 이상
                r.append(word)
                #print(word)
    
    tag_note =  ' '.join(r)
    
    return pd.Series([row['dem_str'], row['note'], tag_note])

In [ ]:
dataframe['note'] = dataframe.dem_title.str.cat(dataframe.dem_note)

In [ ]:
df = dataframe.loc[:,['dem_str','note']]
df.head()

In [ ]:
df[pd.isna(df['note'])==True]
#df = df.fillna('')
#df.isna().sum()

In [ ]:
df = df.apply(kotag, axis=1)
df.head()

In [ ]:
df.columns = ['dem_str','note','tag_note']

In [ ]:
def read_corpus(documents):
    #print(documents)
    for i, plot in enumerate(documents):
        yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(plot, max_len=500), [i])

In [ ]:
train_corpus = list(read_corpus(df.tag_note))

In [ ]:
train_corpus[0:10]

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(size=100, min_count=2, epochs=55)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
model.save_word2vec_format('./output/simpan_tensor.w2v', doctag_vec=True, word_vec=False)

In [ ]:
%run /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/scripts/word2vec2tensor.py -i ./output/simpan_tensor.w2v -o ./output/simpan_plot

In [ ]:
with open('./output/simpan_plot_metadata.tsv','w+', encoding='UTF8') as w:
    w.write('dem_str\tdem_judgement\n')
    for i,j in zip(dataframe.dem_str, dataframe.dem_judgement):
        w.write("%s\t%s\n" % (i,j))

In [ ]:
model.most_similar('법률')